## Automate report building

- Created PDF template on canva
- Using Python, read the template and edit the data for every score in an examination
- attach reports in email and send to students using python


In [1]:
import pandas as pd
import openpyxl
from openpyxl.styles import PatternFill
from openpyxl.styles.colors import Color
import pandas as pd
import os
import comtypes.client

from reportlab.pdfgen.canvas import Canvas
from pdfrw import PdfReader
from pdfrw.buildxobj import pagexobj
from pdfrw.toreportlab import makerl
from reportlab.lib.colors import HexColor
import PyPDF2
import shutil
import textwrap


#for email
import email, smtplib, ssl

from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [ ]:

def get_details(data,e , specialization1 = None, specialization2 = None):
    final_df = data.copy()
    
    f = final_df[final_df.enrollment == e]
    
    campus = f.campus_name.iloc[0].upper()
    name= f.student_name.iloc[0].upper()
    email = f.sunstone_email.iloc[0].lower()
    
    x = f.Specialization.to_list()
    x = set(x)
    
    if 'Core' in x:
        core_score = f[f.Specialization == 'Core'].Score.iloc[0]
        x.remove('Core')
    else:
        core_score = "NA"
        
    if "EC" in x:
        ec_score  = f[f.Specialization == 'EC'].Score.iloc[0]
        x.remove('EC')
    
    else:
        ec_score = "NA"
        
    x = list(x)
    
    if len(x) >= 2:
        if specialization1:
            sp1 = specialization1
        else:
            sp1 = x[0]
            
        if specialization2:
            sp2 = specialization2
        else:
            sp2 = x[1]
    
    elif len(x) == 1:
        
        sp1 = x[0]
        sp2 = "Not Attempted"
        
    elif len(x) == 0:
        sp1 = "Not Attempted"
        sp2 = "Not Attempted"
        
        
    if sp1 != "Not Attempted":
        fs1 = f[f.Specialization == sp1]

        if fs1.shape[0] == 3:

            sp1_subject_list = fs1.Subject.to_list()
            sp1_score_list = fs1.Score.to_list()

        elif fs1.shape[0] == 2:
            sp1_subject_list = fs1.Subject.to_list()
            sp1_subject_list.append("Not Attempted")

            sp1_score_list = fs1.Score.to_list()
            sp1_score_list.append("NA")

        elif fs1.shape[0] == 1:
            sp1_subject_list = fs1.Subject.to_list()
            sp1_subject_list.append("Not Attempted")
            sp1_subject_list.append("Not Attempted")

            sp1_score_list = fs1.Score.to_list()
            sp1_score_list.append("NA")
            sp1_score_list.append("NA")
    
    else:
        sp1_subject_list = ["","",""]
        sp1_score_list = ["","",""]
        
        
    if sp2 != "Not Attempted":
        fs2 = f[f.Specialization == sp2]

        if fs2.shape[0] == 3:

            sp2_subject_list = fs2.Subject.to_list()
            sp2_score_list = fs2.Score.to_list()

        elif fs2.shape[0] == 2:
            sp2_subject_list = fs2.Subject.to_list()
            sp2_subject_list.append("Not Attempted")

            sp2_score_list = fs2.Score.to_list()
            sp2_score_list.append("NA")

        elif fs2.shape[0] == 1:
            sp2_subject_list = fs2.Subject.to_list()
            sp2_subject_list.append("Not Attempted")
            sp2_subject_list.append("Not Attempted")

            sp2_score_list = fs2.Score.to_list()
            sp2_score_list.append("NA")
            sp2_score_list.append("NA")

    else:
        sp2_subject_list = ["","",""]
        sp2_score_list = ["","",""]
        
        
        
    return {"user_id" : e, "Campus" :campus, "Name" :name , "Email" : email,  "Sp1":sp1, "Sp2":sp2,
           "core_score": core_score, "EC_score": ec_score, "SP1_subjects" :sp1_subject_list , "SP1_scores" : sp1_score_list,
            "SP2_subjects" :sp2_subject_list, "SP2_scores":sp2_score_list}

In [ ]:
class generate_report:
    
    def fill_details(self,d):

        user_dict = d

        self.u = user_dict['user_id']
        self.Program = "MBA"
        self.Batch = "2021-2023"
        self.Campus = user_dict['Campus']
        self.Name = user_dict['Name']
        self.Email = user_dict['Email']
        self.Spclz1Name = user_dict["Sp1"]
        self.Spclz2Name = user_dict["Sp2"]
        self.ATM_score = str(user_dict["core_score"])
        self.EC_score = str(user_dict["EC_score"])

        Splz1_subjnameslist  = user_dict["SP1_subjects"]
        Splz1_scoreslist = user_dict["SP1_scores"]
        Splz1_scoreslist = [str(i) for i in Splz1_scoreslist]
        
        Splz2_subjnameslist = user_dict["SP2_subjects"]
        Splz2_scoreslist = user_dict["SP2_scores"]
        Splz2_scoreslist = [str(i) for i in Splz2_scoreslist]

        self.sp1_row1 = {'Name':Splz1_subjnameslist[0] , 'Score':Splz1_scoreslist[0] }
        self.sp1_row2 = {'Name':Splz1_subjnameslist[1] , 'Score':Splz1_scoreslist[1] }
        self.sp1_row3 = {'Name':Splz1_subjnameslist[2] , 'Score':Splz1_scoreslist[2] }

        self.sp2_row1 = {'Name':Splz2_subjnameslist[0] , 'Score':Splz2_scoreslist[0] }
        self.sp2_row2 = {'Name':Splz2_subjnameslist[1] , 'Score':Splz2_scoreslist[1] }
        self.sp2_row3 = {'Name':Splz2_subjnameslist[2] , 'Score':Splz2_scoreslist[2] }

        
    
    def set_alignments(self):
        #warning
        self.X1_textlimit18 = 10
        self.X2_textlimit15 = 20
        self.X3_textlimit14 = 35
        
        # top layer Y
        self.Y1 = 700
        self.Y2 = 617
        self.Y3 = 540
        
        self.Y4mid = 245
        self.Y4top = self.Y4mid + 45
        self.Y4low = self.Y4mid - 45

        self.Y5top = self.Y4low - 45
        self.Y5mid = self.Y5top - 45
        self.Y5low = self.Y5mid - 45
        self.Yec = self.Y4top+45
        self.Yatm = self.Yec + 45
        
        
        # X 1st layer
        self.X1Box1 = 118
        self.X1Box2 = 298
        self.X1Box3 = 477
        
        self.X2Box1 = 150
        self.X2Box2 = 438
        
        self.X3Box1 = self.X2Box1
        self.X3Box2 = self.X2Box2
        
        self.S1NameLoc = self.X2Box1 - 35
        self.S2NameLoc = self.S1NameLoc
        
        self.S1Box2 = self.X1Box2 + 27
        self.S1Box3 = self.S1Box2 + 188
    
    
    def set_outfile(self,outfile_path =None):
        #self.outpath = "x.pdf"
        if outfile_path:
            self.outpath = outfile_path + "/" +"Baseline_" + str(self.u) + ".pdf"
            
        else:
            self.outpath = f"reports/Baseline_{self.u}.pdf"

    
    #load the template
    
    def load_template(self,temp_loc = "template2.pdf"):
        self.template_p1 = PdfReader(temp_loc, decompress=False).pages[0]
        self.template_p2 = PdfReader(temp_loc, decompress=False).pages[1]
        self.template_obj = pagexobj(self.template_p1)
        
        
    def set_text_in_place(self,Cursor,X,Y, text,text_limit, Ymargin_forwrap = 7, wrap_split = None):
        
        if text == "Not Attempted":
            Cursor.setFillColor(HexColor('#eb9800'))
            
        if len(text) <= text_limit:        
            Cursor.drawCentredString(X,Y,text)
            
        else:
            if wrap_split:
                split = text.split(wrap_split)
                
                if len(split) ==1:
                    new_text1 = text[:text_limit-2]
                    new_text2 = text[text_limit-2:]

                    Cursor.drawCentredString(X,Y+Ymargin_forwrap , new_text1)
                    Cursor.drawCentredString(X,Y-Ymargin_forwrap , new_text2)
                    
                else:
                    new_text1, new_text2 = "",""

                    for s in split:
                        new_s =  new_text1+ " " +s

                        if len(new_s) <= text_limit:
                            new_text1 = new_text1+" " +s
                            new_text1 = new_text1.strip()
                        else:
                            new_text2 =  new_text2 +" "+ s
                            new_text2 =  new_text2.strip()     
                    Cursor.drawCentredString(X,Y+Ymargin_forwrap , new_text1)
                    Cursor.drawCentredString(X,Y-Ymargin_forwrap , new_text2)
                
            else:
                new_text1 = text[:text_limit-2]
                new_text2 = text[text_limit-2:]
                                        
                Cursor.drawCentredString(X,Y+Ymargin_forwrap , new_text1)
                Cursor.drawCentredString(X,Y-Ymargin_forwrap , new_text2)
                
        if text == "Not Attempted":
            Cursor.setFillColor(HexColor('#22366A'))
                 
        
    def create_page1(self):
        
        self.set_alignments()
        canvas = Canvas(self.outpath)
        xobj_name = makerl(canvas, self.template_obj)
        canvas.doForm(xobj_name)
        
        # all white color names
        canvas.setFont('Helvetica', 18)
        canvas.setFillColor(HexColor('#ffffff')) 
        
        canvas.drawCentredString(self.X1Box1, self.Y1, "MBA")
        canvas.drawCentredString(self.X1Box2, self.Y1, "2021-2023")
        #campus
        self.set_text_in_place(Cursor=canvas, X=self.X1Box3, Y=  self.Y1,text=self.Campus, 
                               text_limit = self.X1_textlimit18,wrap_split = " ")
        
        #all blue 
        canvas.setFont('Helvetica', 15)
        canvas.setFillColor(HexColor('#22366A'))
        #name
        self.set_text_in_place(Cursor=canvas, X=self.X2Box1, Y=  self.Y2,text=self.Name, 
                       text_limit = self.X2_textlimit15,wrap_split = " ")
        #email
        self.set_text_in_place(Cursor=canvas, X=self.X2Box2, Y=  self.Y2,text=self.Email, 
               text_limit = self.X2_textlimit15+5,wrap_split = None)
        
        #Specialization1
        self.set_text_in_place(Cursor=canvas, X=self.X3Box1, Y=  self.Y3,text=self.Spclz1Name, 
               text_limit = self.X2_textlimit15,wrap_split = " ")
        
        #specialization2
        self.set_text_in_place(Cursor=canvas, X=self.X3Box2, Y=  self.Y3,text=self.Spclz2Name, 
               text_limit = self.X2_textlimit15,wrap_split = " ")
        
        
        #add details in table
        canvas.setFontSize(14)
        
        #Specialization1
        self.set_text_in_place(Cursor=canvas, X=self.S1NameLoc, Y=  self.Y4mid,text=self.Spclz1Name, 
               text_limit = self.X2_textlimit15,wrap_split = " ")
        
        #specialization2
        self.set_text_in_place(Cursor=canvas, X=self.S1NameLoc, Y=  self.Y5mid,text=self.Spclz2Name, 
               text_limit = self.X2_textlimit15,wrap_split = " ")
        
        
        #spclzation1 Sub 1,2,3 Name
        self.set_text_in_place(Cursor=canvas, X=self.S1Box2, Y= self.Y4top,text=self.sp1_row1['Name'], 
                               text_limit = self.X3_textlimit14,wrap_split = " ")
                
        self.set_text_in_place(Cursor=canvas, X=self.S1Box2, Y= self.Y4mid,text=self.sp1_row2['Name'], 
                               text_limit = self.X3_textlimit14,wrap_split = " ")
        
        self.set_text_in_place(Cursor=canvas, X=self.S1Box2, Y= self.Y4low,text=self.sp1_row3['Name'], 
                               text_limit = self.X3_textlimit14,wrap_split = " ")
        
        
        
        #spclzation2 Sub 1,2,3 Name
        self.set_text_in_place(Cursor=canvas, X=self.S1Box2, Y= self.Y5top,text=self.sp2_row1['Name'], 
                               text_limit = self.X3_textlimit14,wrap_split = " ")
                
        self.set_text_in_place(Cursor=canvas, X=self.S1Box2, Y= self.Y5mid,text=self.sp2_row2['Name'], 
                               text_limit = self.X3_textlimit14,wrap_split = " ")
        
        self.set_text_in_place(Cursor=canvas, X=self.S1Box2, Y= self.Y5low,text=self.sp2_row3['Name'], 
                               text_limit = self.X3_textlimit14,wrap_split = " ")
    
        
        
        #add scores

        canvas.drawCentredString(self.S1Box3, self.Yatm, self.ATM_score)
        canvas.drawCentredString(self.S1Box3, self.Yec, self.EC_score)
        canvas.drawCentredString(self.S1Box3, self.Y4top, self.sp1_row1['Score'])
        canvas.drawCentredString(self.S1Box3, self.Y4mid, self.sp1_row2['Score'])
        canvas.drawCentredString(self.S1Box3, self.Y4low, self.sp1_row3['Score'])
        canvas.drawCentredString(self.S1Box3, self.Y5top, self.sp2_row1['Score'])
        canvas.drawCentredString(self.S1Box3, self.Y5mid, self.sp2_row2['Score'])
        canvas.drawCentredString(self.S1Box3, self.Y5low, self.sp2_row3['Score'])
        
        canvas.save()
        self.combine_temp2_with_file()
        
        
        
    def combine_temp2_with_file(self):
        file1 = open(self.outpath,'rb')
        file2 = open(f"temp_page2.pdf",'rb')
        
        pdfReader = PyPDF2.PdfFileReader(file1)
        pdfReader2 = PyPDF2.PdfFileReader(file2)
        pdfWriter = PyPDF2.PdfFileWriter()
        
        pdfWriter.add_page(pdfReader.getPage(0))
        pdfWriter.add_page(pdfReader2.getPage(0))
        
        
        newFile = open(f"final_reports/Baseline_{self.u}.pdf", 'wb')
        
        pdfWriter.write(newFile)
        file1.close()
        file2.close()
        newFile.close()
        
        print(f"pdf generated for user {self.u} - {self.Name}")

In [ ]:
def send_email(id_,sender,pw,email_text, receiver ,bcc=None):
    
    subject = "BME Baseline Score Card"
    body = email_text
    sender_email = sender
    receiver_email = receiver
    password = pw

    # Create a multipart message and set headers
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject
    message["CC"] = bcc  # Recommended for mass emails

    # Add body to email
    
    message.attach(MIMEText(body, "plain"))

    filename = f"final_reports/Baseline_{id_}.pdf"  # In same directory as script

    # Open PDF file in binary mode
    with open(filename, "rb") as attachment:
        # Add file as application/octet-stream
        # Email client can usually download this automatically as attachment
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())

    # Encode file in ASCII characters to send by email    
    encoders.encode_base64(part)

    # Add header as key/value pair to attachment part
    part.add_header(
        "Content-Disposition",
        f"attachment; filename= {filename}",
    )

    # Add attachment to message and convert message to string
    message.attach(part)
    text = message.as_string()
    #try:

        # Log in to server using secure context and send email
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, text)
    print(f'email successfully triggered for {id_}')
#except:
     #   print(f'email execution error for {id_}')